In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

In [2]:
from formulas_auxiliares_ajuste_base import months_dict, dict_contas
from class_ajusta_batimento_email import *

In [3]:
path = '\\\\DRIOC0231PFS\\Apoio_SB\\Geral\\BD\\Aluguel_Offshore\\Arquivos diários\\jul2018\\consolidado\\'
file = 'bases_batimento_pb_calypso.xlsx'
arquivo = path+file

In [4]:
pb = pd.read_excel(arquivo, sheetname='base_pb')
calypso = pd.read_excel(arquivo, sheetname='base_calypso')
ticker_diffs = pd.read_excel(arquivo, sheetname='ticker_diffs')

##### Distribuição intermediária (somente quantidade. não abre os books)

In [5]:
calypso_month = calypso.drop(['month', 'year'], axis=1).reset_index(drop=True).copy()
pb_month = pb.drop(['month', 'year'], axis=1).reset_index(drop=True).copy()
pb_month['acc'] = pb_month['conta'].map(dict_contas)
calypso_month['acc'] = calypso_month['conta'].map(dict_contas)

In [6]:
dias_calypso = calypso_month[['acc', 'data_ref']].copy()
dias_calypso['tipo'] = 'calypso'
dias_pb = pb_month[['acc', 'data_ref']].copy()
dias_pb['tipo'] = 'pb'
dias_ok = pd.concat([dias_calypso, dias_pb], axis=0)
dias_ok['check'] = 1
pvt = pd.pivot_table(dias_ok, values=['check'],
                     index='data_ref', columns=['tipo','acc'])

In [7]:
if pvt.shape!=pvt.dropna().shape:
    print('Nem todos os arquivos estão disponíveis!')
    print(pvt)
    raise error

In [8]:
#Agrupa as informações que estão no calypso eliminando books. Junta em conta_pb, agent, data_ref, ticker e currency
calypso_batimento = calypso_month.groupby(by=['conta_pb','agent','data_ref','ticker','currency']).sum().copy().reset_index()
calypso_batimento['acc'] = calypso_batimento['conta_pb'].map(dict_contas)
calypso_batimento = calypso_batimento[calypso_batimento['quantidade']<0].reset_index(drop=True)
calypso_batimento = calypso_batimento.groupby(by=['acc','data_ref', 'ticker','currency']).sum().reset_index()  
#é somente um indice para poder checar quais informações do calypso não passaram pelo filtro da PB
calypso_batimento['id'] = range(1, calypso_batimento.shape[0] +1)
pb_month = pb_month.groupby(by=['acc','data_ref','ticker','currency']).sum().reset_index()

In [9]:
calypso_bate_pb = pd.merge(pb_month, calypso_batimento, on=['acc','data_ref','ticker','currency'],  how='left')

In [10]:
check_2 = calypso_bate_pb.shape[0]-pb_month.shape[0]
if check_2!=0:
    print ('%s linhas foram duplicadas durante o merge. Favor verificar se houve algo de errado' %check_2)

In [11]:
# Sobra de destino são os tickers que estavam no pb, mas não tinham correspondência no Calypso
sobra_destino = calypso_bate_pb[(calypso_bate_pb['quantidade'].isnull()==True)&(calypso_bate_pb['accrual']!=0)]

In [12]:
calypso_bate_pb = calypso_bate_pb[calypso_bate_pb['quantidade'].isnull()==False]

In [13]:
# Sobra de origem são os tickers que estavam no Calypso, mas não tinham correspondência no pb
sobra_origem = calypso_batimento[~calypso_batimento['id'].isin(calypso_bate_pb['id'].dropna().drop_duplicates())]

In [14]:
pct_in_pb_not_in_calypso = (1-calypso_bate_pb.shape[0]/pb_month.shape[0])*100
pct_in_calypso_not_in_pb=calypso_batimento[~calypso_batimento['id'].isin(calypso_bate_pb['id'].dropna().drop_duplicates())].shape[0]/\
                                calypso_batimento.shape[0]*100 
texto_1 = '%s pct dos trades estão no prime broker, mas não acharam correspondência no Calypso' % pct_in_pb_not_in_calypso
texto_2 = '%s pct dos trades foram achados no calypso, mas não têm correspondência no prime broker' % pct_in_calypso_not_in_pb

print (texto_1)
print(texto_2)

3.759398496240607 pct dos trades estão no prime broker, mas não acharam correspondência no Calypso
0.7751937984496124 pct dos trades foram achados no calypso, mas não têm correspondência no prime broker


In [15]:
batido = calypso_bate_pb[calypso_bate_pb['qt']==calypso_bate_pb['quantidade']].copy()
batido['status']='batido'
divergentes = calypso_bate_pb[calypso_bate_pb['qt']!=calypso_bate_pb['quantidade']].copy()
corrigidos = divergentes[((divergentes['ticker'].isin(ticker_diffs['ticker']))&(divergentes['acc']!='ubs_cayman'))].copy()
corrigidos['status'] = 'corrigido'
n_batidos = divergentes[~((divergentes['ticker'].isin(ticker_diffs['ticker']))&(divergentes['acc']!='ubs_cayman'))].copy()
n_batidos['status']='n_batido'
distribuidos = pd.concat([batido, corrigidos, n_batidos], axis=0)

In [16]:
writer_1 = pd.ExcelWriter(path+'distribuicao_intermediaria_por_quantidade.xlsx')

In [17]:
batido.to_excel(writer_1, sheet_name='trades_batidos', index=False)
divergentes.to_excel(writer_1, sheet_name='trades_divergentes', index=False)
corrigidos.to_excel(writer_1, sheet_name='trades_corrigidos', index=False)
n_batidos.to_excel(writer_1, sheet_name='trades_n_batidos', index=False)
distribuidos.to_excel(writer_1, sheet_name='trades_distribuidos', index=False)
writer_1.save()

In [18]:
# Fazer qualquer alteração necessária na planilha entre esses dois códigos!!

In [19]:
batido= pd.read_excel(path+'distribuicao_intermediaria_por_quantidade.xlsx', sheetname='trades_batidos')
divergentes= pd.read_excel(path+'distribuicao_intermediaria_por_quantidade.xlsx', sheetname='trades_divergentes')
corrigidos= pd.read_excel(path+'distribuicao_intermediaria_por_quantidade.xlsx', sheetname='trades_corrigidos')
n_batidos= pd.read_excel(path+'distribuicao_intermediaria_por_quantidade.xlsx', sheetname='trades_n_batidos')
distribuidos= pd.read_excel(path+'distribuicao_intermediaria_por_quantidade.xlsx', sheetname='trades_distribuidos')

#### Distribuição por books

In [20]:
# em algum momento dessa parte eu vou ter que tirar o que é TREAS_OPS

In [21]:
calypso_month_books = calypso.copy()

In [22]:
calypso_month_books = pd.merge(calypso_month_books, calypso_batimento.drop('id', axis=1), \
                               on=['acc','data_ref','ticker','currency'], how='left').rename(columns={
    'quantidade_x':'quantidade_book',
    'quantidade_y':'quantidade_ticker'
})

In [23]:
check_3 = calypso_month_books.shape[0]-calypso.shape[0]
if check_3 != 0:
    print ('%s linhas foram adicionadas durante o merge. Favor verificar quais erros aconteceram.' %check_3)

In [24]:
calypso_month_books = calypso_month_books[calypso_month_books['quantidade_ticker'].isnull()==False]

In [25]:
check_4 = calypso_month_books.shape[0]-calypso.shape[0]
if check_4 != 0 :
    print ('%s trades foram perdidos durante o merge. Favor ferificar quais erros aconteceram.' %check_4)

In [26]:
calypso_month_books[calypso_month_books.book.str.contains('TREAS')]

,conta_pb,conta,agent,data_ref,quantidade_book,book,ticker,isin,cusip,sedol,currency,month,year,acc,quantidade_ticker
17,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-1,TREAS_OPS.CAYMAN,BSMXB MM,MX41BS060005,NaN,BF1FXN2,MXN,jul,2018,ubs_cayman,-1
76,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-11360,TREAS_OPS.CAYMAN,KHC US,US5007541064,500754106,BYRY499,USD,jul,2018,ubs_cayman,-11360
81,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-2214,TREAS_OPS.CAYMAN,BSAC US,US05965X1090,05965X109,2000257,USD,jul,2018,ubs_cayman,-2214
116,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-33746,TREAS_OPS.CAYMAN,RGR US,US8641591081,864159108,2856289,USD,jul,2018,ubs_cayman,-33746
121,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-658556,TREAS_OPS.CAYMAN,JCP US,US7081601061,708160106,2680303,USD,jul,2018,ubs_cayman,-658556
122,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-18883,TREAS_OPS.CAYMAN,HSY US,US4278661081,427866108,2422806,USD,jul,2018,ubs_cayman,-18883
161,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-1257642,TREAS_OPS.CAYMAN,EWW US,US4642868222,464286822,2373818,USD,jul,2018,ubs_cayman,-1294302
164,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-32797,TREAS_OPS.CAYMAN,ECH US,US4642866408,464286640,B29FC81,USD,jul,2018,ubs_cayman,-42727
168,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-112931,TREAS_OPS.CAYMAN,XLP US,US81369Y3080,81369Y308,2369389,USD,jul,2018,ubs_cayman,-112931
169,75202215,ASSET,UBS SECURITIES LLC,2018-07-02,-101624,TREAS_OPS.CAYMAN,SPY US,US78462F1030,78462F103,2840215,USD,jul,2018,ubs_cayman,-101624


In [30]:
calypso_month_books[(calypso_month_books['isin']=='US8641591081')&\
                    (calypso_month_books.data_ref==pd.to_datetime('2018-07-10'))]

,conta_pb,conta,agent,data_ref,quantidade_book,book,ticker,isin,cusip,sedol,currency,month,year,acc,quantidade_ticker
2046,75202215,ASSET,UBS SECURITIES LLC,2018-07-10,-36485,TREAS_OPS.CAYMAN,RGR US,US8641591081,864159108,2856289,USD,jul,2018,ubs_cayman,-36485


In [29]:
calypso_month_books[calypso_month_books.book=='TREAS_OPS.CAYMAN']['quantidade_book'].sum()

-86536908

In [26]:
# Aqui a gente está adicionando a coluna de Accrual pros trades do calyspo
calypso_month_books = pd.merge(calypso_month_books, distribuidos, on=['acc','data_ref','ticker','currency'], how='left')
calypso_month_books = calypso_month_books[calypso_month_books['accrual'].isnull()==False]

In [27]:
check_5 = calypso_month_books.shape[0]-calypso.shape[0]
print ('%s trades não foram encontrados durante a distribuição. Eles provavelmente não foram batidos.' %check_5)

-57 trades não foram encontrados durante a distribuição. Eles provavelmente não foram batidos.


In [28]:
calypso_month_books = calypso_month_books[calypso_month_books['quantidade_book']!=0]
check_6 = calypso.shape[0]-calypso_month_books.shape[0]
print ('''%s linhas foram retiradas da base por terem quantidade zerada. Ainda restam %s linhas de operações.
Ajustes na base do calypso estão prontos. Dados estão prontos para serem distribuídos entre os books.''' \
                                                                       %(check_6, calypso_month_books.shape[0]))

57 linhas foram retiradas da base por terem quantidade zerada. Ainda restam 3129 linhas de operações.
Ajustes na base do calypso estão prontos. Dados estão prontos para serem distribuídos entre os books.


In [29]:
por_conta = calypso_month_books.groupby(by=['conta_pb','data_ref','ticker','currency']).sum().reset_index()

In [30]:
# Juntando a quantidade por sub_account do pb para poder dropar as contas positivas (que estão net doadas)
calypso_month_books = pd.merge(calypso_month_books, por_conta.drop(['year', 'qt', 'accrual',\
                                                                   'quantidade', 'id', 'quantidade_ticker'], axis=1), \
                                                       on=['conta_pb','data_ref','ticker','currency'], how='left')\
.rename(columns={
    'quantidade_book_x':'quantidade_book',
    'quantidade_book_y':'quantidade_conta_pb'
})

In [31]:
calypso_month_books = calypso_month_books[calypso_month_books['quantidade_conta_pb']<0]

In [32]:
calypso_month_books = calypso_month_books[calypso_month_books['accrual'].isnull()==False]
print ('Foram feitos os casamentos de trade por sub_account')

Foram feitos os casamentos de trade por sub_account


In [33]:
abertura = calypso_month_books[calypso_month_books['id'].isin(distribuidos['id'].drop_duplicates())==True]
trades_out_abertura = calypso_month_books[calypso_month_books['id'].isin(distribuidos['id'].drop_duplicates())==False]
if trades_out_abertura.shape[0]!=0:
    print('ATENÇãO!!! %s trades foram encontrados em ambas as bases, porém não foram distribuídos.'%trades_out_abertura.shape[0])

In [34]:
abertura = abertura.drop(['isin', 'cusip','month', 'year', 'quantidade', 'id'], axis=1)

In [35]:
abertura['percent'] = abertura['quantidade_book']/abertura['quantidade_ticker']

In [36]:
check_7 = abertura.groupby(by=['acc','data_ref','ticker','currency']).sum().reset_index()
check_7 = check_7[np.isclose(check_7['percent'], 1)==False].shape[0]
if check_7 != 0:
    print ('O batimento não foi feito!!! %s contas somam valor diferente a 100\%')

In [37]:
abertura['accrual_final'] = abertura['percent']*abertura['accrual']

In [38]:
abertura = abertura.drop(['conta_pb', 'quantidade_ticker', 'qt', 'accrual','quantidade_conta_pb', 
                          'percent'], axis=1).reset_index(drop=True)

In [39]:
# Tickers que estão batidos ou que foram corrigidos. Todos distribuídos
abertura_batidos = abertura[abertura['status']!='n_batido']
# Tickers que NÃO ESTÃO BATIDOS EM QUANTIDADE, mas que foram distribuídos, pois foram encontrados em ambas as bases
abertura_n_batidos = abertura[abertura['status']=='n_batido']

In [40]:
bate_soma = abertura.groupby(by=['acc', 'currency']).sum().reset_index()
bate_soma_pb = pb_month.groupby(by=['acc', 'currency']).sum().reset_index()

In [41]:
check_8 = pd.merge(bate_soma_pb, bate_soma, on=['acc', 'currency'], how='left').rename(columns={
'qt':'quantidade_pb',
'accrual':'custos_pb',
'quantidade_book':'quantidade_calypso',
'accrual_final':'custos_batidos'
})
check_8['bate_qt'] = check_8['quantidade_pb']-check_8['quantidade_calypso']
check_8['bate_custos'] = check_8['custos_pb']-check_8['custos_batidos']
check_8['bate_custos_pct'] = check_8['custos_pb']/check_8['custos_batidos']
check_8 = check_8[['acc', 'currency', 'quantidade_pb', 'quantidade_calypso', 'bate_qt', 'custos_pb', 'custos_batidos',
      'bate_custos', 'bate_custos_pct']]   

In [42]:
check_9 = check_8[(check_8['bate_custos_pct']<0.95)|(check_8['bate_custos_pct']>1.05)].shape[0]
if check_9!=0:
    print('%s contas tiveram diferença maior do que 5pct do valor do prime broker' %check_9)

1 contas tiveram diferença maior do que 5pct do valor do prime broker


In [43]:
check_8 = check_8.drop(['quantidade_pb','quantidade_calypso','bate_qt'],axis=1)

In [44]:
## Aqui você copia as informações que vêm do e-mail de Cayman!
info_emails = pd.DataFrame({
    'acc':['ms_cayman','ms_cayman','ms_cayman','ms_cayman','ubs_cayman','int_c','int_c','int_c'],
    'currency':['USD','EUR','GBP','MXN','MXN','AUD','USD','MXN'],
    'accrual_final':[-151302.90,-4.18,-1.20,-1774308.52,-993668.89,-2588.31,-23637.36,-289.12]
})

In [45]:
info_emails

,acc,accrual_final,currency
0,ms_cayman,-151302.90,USD
1,ms_cayman,-4.18,EUR
2,ms_cayman,-1.20,GBP
3,ms_cayman,-1774308.52,MXN
4,ubs_cayman,-993668.89,MXN
5,int_c,-2588.31,AUD
6,int_c,-23637.36,USD
7,int_c,-289.12,MXN


In [46]:
abertura_batidos_ajustado, abertura_batidos_consol, ajuste = \
                            AjustaBatimentoEmailCayman(path).ajuste_dados_email(info_emails, abertura_batidos)

1 linhas foram dropadas da base, porque os valores não foram descontados no conta.
454 tickers_book foram dropados da abertura porque suas contas não faziam sentido com o que foi cobrado


In [47]:
writer_2 = pd.ExcelWriter(path+'distribuicao_asset_consolidada_ajustada_email_cayman.xlsx')

In [48]:
abertura_batidos_ajustado.to_excel(writer_2, sheet_name='trades_batidos_ajustados', index=False)
abertura_batidos_consol.to_excel(writer_2, sheet_name='trades_abertura_batidos_consol', index=False)
check_8.to_excel(writer_2, sheet_name='check_distribuicao_igual_pb', index=False)
abertura.to_excel(writer_2, sheet_name='tickers_abertura', index=False)
abertura_batidos.to_excel(writer_2, sheet_name='tickers_batidos_corrigidos', index=False)
abertura_n_batidos.to_excel(writer_2, sheet_name='n_batidos_mas_distribuidos', index=False)
ajuste.to_excel(writer_2, sheet_name='ajuste', index=False)
writer_2.save()